modelo.
voy a usar 2 datasets, los datos limpios que tengo exportados
y el csv de keywords

In [8]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [ ]:
# importo set original
#df_movies_orig = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\Dataset\movies_dataset.csv", parse_dates = ['release_date'])

In [9]:
# importo set limpio 
columnas = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity']
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv", usecols = columnas)

In [ ]:
df_movies.shape

In [10]:
columnas = ['popularity', 'release_year']  # Nombres de las columnas por las cuales deseas ordenar
df_movies = df_movies.sort_values(by=columnas, ascending=False)

In [ ]:
df_movies.shape

In [ ]:
df_movies.head(50)

In [11]:
# elimino cortos y series
df_movies = df_movies[(df_movies['runtime'] == 0.0) | (df_movies['runtime']>40)]

In [ ]:
df_movies.head(50)

In [ ]:
df_movies.shape

In [12]:
df_movies['genres'] = df_movies['genres'].str.replace("[\[\]',]", "", regex=True)
#df_movies['cast'] = df_movies['cast'].str.replace("[\[\]',]", "", regex=True)


IMPORTO KEYWORDS

In [ ]:
df_keywords = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\DATA SETS\movies kaggle\keywords.csv")

In [ ]:
duplicados_m = df_keywords[df_keywords['id'].duplicated()]
duplicados_m.shape


In [ ]:
df_keywords = df_keywords.drop_duplicates(subset = 'id')

In [ ]:
df_keywords.shape

desanidar keywords

In [ ]:
def desanidar(obj):
    lista = []
    for i in literal_eval (obj):
        try:
            lista.append(i['name'])
        except (ValueError, TypeError, KeyError):
            pass
    return lista

In [ ]:
df_keywords['keywords'] = df_keywords['keywords'].apply(desanidar)

In [ ]:
df_keywords['keywords'] = df_keywords['keywords'].apply(lambda x: ' '.join(x))

In [ ]:
#print(type(df_keywords['keywords'][0]))

In [ ]:
df_keywords

In [ ]:
#df_movies_orig['genres'] = df_movies_orig['genres'].apply(desanidar)

In [ ]:
#print(type(df_keywords['keywords'][0]))
#print(df_keywords['keywords'][0])
#print(type(df_movies_orig['genres'][0]))
#print(df_movies_orig['genres'][0])


JOIN 

In [ ]:
# con inner me muestra lo que está en los dos
#quedan por fuera registros de mas de keywords, no los necesito
df_movies=df_movies.merge(df_keywords, how = 'inner', on = 'id')
df_movies.shape



REDUZCO LA CANTIDAD DE REGISTROS




In [ ]:
#no hace falta, ya esta ordenada por popularidad y anio
#df_movies = df_movies.sort_values(by='release_year', ascending = False)

In [ ]:
df_movies = df_movies.head(15000)

In [ ]:
#cantidad = 10000  # Cantidad de registros que deseas en el DataFrame reducido
#df_movies = df_movies.sample(n=cantidad, random_state=42)


In [ ]:
#agrego indice

df_movies['index'] = df_movies.reset_index().index

In [ ]:
df_movies



CONSTRUCCION DEL MODELO




In [ ]:
# selecting the relevant features for recommendation

selected_features = ['keywords','title']
print(selected_features)

In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  df_movies[feature] = df_movies[feature].fillna('')

In [ ]:
combined_features = df_movies['keywords']+ ' '+ df_movies['title']

In [ ]:
print(combined_features)

In [ ]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

In [ ]:
df_movies.info()

In [ ]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

In [ ]:
print(similarity.shape)

#### busco similitud por nombre

In [ ]:
#movie_name = "Titanic"
titulo_pelicula= "The Terminator"


In [ ]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = df_movies['title'].tolist()
print(list_of_all_titles)

In [ ]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(titulo_pelicula, list_of_all_titles)
print(find_close_match)

In [ ]:
close_match = find_close_match[0]
print(close_match)

In [ ]:
# finding the index of the movie with title
index_of_the_movie = df_movies[df_movies.title == close_match]['index'].values[0]
print(index_of_the_movie)

In [ ]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

In [ ]:
len(similarity_score)

In [ ]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

In [ ]:
# resultados

print('Peliculas sugeridas : \n')

i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df_movies[df_movies.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',title_from_index)
    i+=1

In [ ]:
print('Movies suggested for you:\n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos
indices = [movie[0] for movie in top_movies]
df_movies[df_movies.index==index]['title'].values[0]
#title_from_index= df_movies.loc[indices, 'title'].values    esto anda tambien
df_movies['rank'] = range(1, 6)
for i, movie_title in enumerate(df_movies['title_from_index'].head(5), 1):
    print(i, '.', movie_title)



In [ ]:
print('Movies suggested for you:\n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos

# Obtener los índices de las películas
indices = [movie[0] for movie in top_movies]

# Agregar la columna 'title_from_index' con los títulos correspondientes
df_movies['title_from_index'] = df_movies.loc[indices, 'title'].values

# Agregar la columna 'rank' para el ranking de las películas
df_movies['rank'] = range(1, 6)

# Imprimir las películas sugeridas
for i, movie_title in enumerate(df_movies['title_from_index'].head(5), 1):
    print(i, '.', movie_title)


In [ ]:
print('Movies suggested for you : \n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos

df_movies['title_from_index'] = df_movies.loc[top_movies[:, 0], 'title'].values  # Agregar la columna 'title_from_index' con los títulos correspondientes

df_movies['rank'] = range(1, 6)  # Agregar la columna 'rank' para el ranking de las películas

# Imprimir las películas sugeridas
df_movies[['rank', 'title_from_index']].apply(lambda x: print(x['rank'], '.', x['title_from_index']), axis=1)


In [ ]:
print(top_movies)